In [1]:
import os
import time
import pyspark
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
home_path = "/mapr/datalake/uhc/ei/pi_ara/sherlock/development/HPP-Provider-Signature-Monitoring/"
try:
    os.chdir(home_path)
except:
    os.mkdir(home_path)

In [3]:
def get_data(query):
    """
    Returns the pySpark DataFrame
    
    Parameters:
        query (str): The input HQL query
        
    Returns:
        DataFrame (pySpark)
    """
    try:
        return sessn.sql(query)
    except Exception as e:
        print(e)

In [4]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

name 'sessn' is not defined
'SparkContext' object has no attribute 'SparkContext'


In [5]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '12g'), 
                                   ('spark.executor.instances', '15'),
                                   ('spark.executor.cores', '6'), 
                                   ('spark.driver.cores','4'), 
                                   ('spark.driver.memory','16g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                   ('spark.yarn.executor.memoryOverhead', '2g')])
sessn = SparkSession.builder.appName("HPP").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

### Provider Signature

In [24]:
# Input 
tin_list_table = "pi_usrs.hpp_ps_tin_list"
# Output
raw_table = "pi_usrs.hpp_ps_raw_table"
# Pulling data from Dec'18 to Jan'20
lower_dt  = "20181201"
upper_dt  = "20200131"

#### Data pull from 837p

In [8]:
query = """
SELECT 
  * 
FROM 
  (
    SELECT 
      DISTINCT 
      srk_clm_id,  
      clm_bill_prov_npi,
      cl_rend_prov_npi, 
      CASE WHEN trim(clm_bill_prov_tin) = '' OR trim(clm_bill_prov_tin) IS NULL THEN trim(clm_bill_prov_ssn) ELSE trim(clm_bill_prov_tin) END AS prov_tin,
      batch_run_dt, 
      cl_proc_cd, 
      cl_proc_mod1, 
      cl_proc_mod2, 
      cl_proc_mod3, 
      cl_line_item_chrg_amt, 
      cl_place_of_srcv, 
      cl_emergency_ind, 
      cl_units_billed 
    FROM 
      pi_prod.837p_clm_analytic_data 
    WHERE
      batch_run_dt BETWEEN '{}' AND '{}'
  ) AS a 
  INNER JOIN {} AS b ON a.prov_tin = trim(b.Tin)
""".format(lower_dt, upper_dt, tin_list_table)

In [10]:
query = " ".join(query.split())

In [12]:
t1 = time.time()
data = get_data(query)
data.write.mode("overwrite").format("orc").saveAsTable(raw_table)
t2 = time.time()

#### Sanity Check

In [25]:
data = sessn.table(raw_table)

In [26]:
data_cnt = data.count()
data_cnt

1251531

In [27]:
dt_chk = data.select(F.min("batch_run_dt"), F.max("batch_run_dt")).toPandas()
dt_chk

min(batch_run_dt) max(batch_run_dt)
0          20191101          20200331

In [28]:
uniq_prov_tin = data.select("prov_tin").distinct().toPandas()
uniq_prov_tin.shape[0]

795

#### Closing the connection

In [29]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

'SparkContext' object has no attribute 'SparkContext'
